<a href="https://colab.research.google.com/github/lagnajit007/Music_Note_Gen/blob/main/Music_notes_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import matplotlib.pyplot as plt
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

# Define the music pattern representation
notes = ["SA", "RE", "GA", "MA", "PA", "DHA", "NI"]

# Dynamic pattern length
pattern_length = int(input("Enter the desired pattern length: "))

# User-defined constraints

# required_notes = ["SA", "PA"]
# forbidden_notes = ["DHA"]

# Define the fitness function
def fitness(pattern):
    repeated_notes = 0
    large_jumps = 0
    for i in range(1, len(pattern)):
        if pattern[i] == pattern[i-1]:
            repeated_notes += 1
        if abs(notes.index(pattern[i]) - notes.index(pattern[i-1])) > 2:
            large_jumps += 1
    out_of_range_notes = sum(1 for note in pattern if note not in notes)
    # missing_required_notes = sum(1 for note in required_notes if note not in pattern)
    # has_forbidden_notes = sum(1 for note in forbidden_notes if note in pattern)
    return repeated_notes + out_of_range_notes + large_jumps + #missing_required_notes + has_forbidden_notes

# Define the crossover function
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 2)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Define the mutation function
def mutation(pattern, mutation_rate):
    if random.random() < mutation_rate:
        mutation_point = random.randint(0, len(pattern) - 1)
        new_note = random.choice(notes)
        pattern[mutation_point] = new_note
    return pattern

# Function to maintain diversity in the population
def maintain_diversity(population): # Moved the function definition before it is called.
    unique_population = []
    seen = set()
    for pattern in population:
        pattern_tuple = tuple(pattern)
        if pattern_tuple not in seen:
            unique_population.append(pattern)
            seen.add(pattern_tuple)
    return unique_population

# Initialize the population
population_size = 100
population = [random.sample(notes, min(pattern_length, len(notes))) for _ in range(population_size)]

# Evolution parameters
generations = 100
elitism_count = 2
initial_mutation_rate = 0.1

# Track fitness over generations
fitness_history = []

# Evolve the population
for generation in range(generations):
    # Recalculate fitness scores after maintaining diversity
    fitness_scores = [fitness(pattern) for pattern in population]

    parents = [population[i] for i in sorted(range(len(population)), key=lambda i: fitness_scores[i])[:10]]

    # Create the next generation
    new_population = parents[:elitism_count]  # Elitism

    # Adjust mutation rate
    mutation_rate = initial_mutation_rate / (1 + generation / 10)

    while len(new_population) < population_size:
        parent1 = random.choice(parents)
        parent2 = random.choice(parents)
        child1, child2 = crossover(parent1, parent2)
        child1 = mutation(child1, mutation_rate)
        child2 = mutation(child2, mutation_rate)
        new_population.extend([child1, child2])

    # Maintain diversity
    population = maintain_diversity(new_population)[:population_size]

    # Track and log fitness
    # Recalculate fitness scores after maintaining diversity
    fitness_scores = [fitness(pattern) for pattern in population]
    fitness_history.append(min(fitness_scores))
    best_pattern_index = sorted(range(len(population)), key=lambda i: fitness_scores[i])[0]
    best_pattern = population[best_pattern_index] # Use the index to get the best pattern
    logging.info(f"Generation {generation}: Best pattern: {best_pattern} with fitness: {min(fitness_scores)}")

# Visualization
plt.plot(fitness_history)
plt.xlabel('Generation')
plt.ylabel('Best Fitness Score')
plt.title('Fitness Score Evolution')
plt.show()

# Choose the best pattern as the final result
fitness_scores = [fitness(pattern) for pattern in population]
best_pattern = population[sorted(range(len(population)), key=lambda i: fitness_scores[i])[0]]

# Print the best pattern
print("Best music pattern:", best_pattern)

# Function to maintain diversity in the population
def maintain_diversity(population):
    unique_population = []
    seen = set()
    for pattern in population:
        pattern_tuple = tuple(pattern)
        if pattern_tuple not in seen:
            unique_population.append(pattern)
            seen.add(pattern_tuple)
    return unique_population